In [17]:
# CIS 및 URN 관련 논문 요약하기 (250905)

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [2]:
# 디렉토리 내 모든 파일을 리스트로 변환하는 함수 정의

import os

def list_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# 지정된 디렉토리 내 모든 파일명을 리스트로 호출
file_names = list_files('./data')
print(file_names)

['./data\\constitution_of_Korea.pdf', './data\\doc_CIS_URN_250903.pdf']


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
# 버전 문제 해결 : https://chan7ee.tistory.com/entry/issue-tensorflow-keras-transformer-%EB%B2%84%EC%A0%84-%EB%AC%B8%EC%A0%9C
# pip install tf-keras==2.16.0

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',    # 다국어 모델
    # model_name='jhgan/ko-sroberta-multitask',  # 한국어 모델 - 에러 발생 (250603)
    # model_name = 'BAAI/bge-m3',                # 에러 발생 (250603)
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings_model

C:\Users\wonta\AppData\Local\Temp\ipykernel_15212\881631784.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [5]:
loader = PyMuPDFLoader(file_names[1])       # 폴더 내 파일 1개만 존재 : 여러 개일 경우, 최초 1개 DB 생성 후, Add 방식으로 진행 (250605)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100) 
docs = text_splitter.split_documents(documents)

# 임베딩 DB 생성 : 파일로 저장하지 않으므로, 새로 실행할 경우 초기화됨 (250605)
db = Chroma.from_documents(
    documents=docs, embedding=embeddings_model, collection_name="db_URN_CIS"
)

In [6]:
# 파일로 DB 저장하기

from langchain.vectorstores import Chroma

# 'store_db' 경로 생성
vectorstore_path = 'store_db'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
store_db = Chroma.from_documents(docs, embeddings_model, persist_directory=vectorstore_path, collection_name="db_URN_CIS_250905")

# 벡터스토어 데이터를 디스크에 저장
store_db.persist()

print("Vector created and persisted.")

Vector created and persisted.


C:\Users\wonta\AppData\Local\Temp\ipykernel_23536\1364977411.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  store_db.persist()


In [24]:
# 폴더에 여러 개의 pdf 파일이 있는 경우, 
# 데이터베이스 추가하기 : 폴더 내부의 파일 전체를 DB 처리

'''
for i in range(len(file_names)-1):
    loader = PyMuPDFLoader(file_names[i+1])
    pages_new = loader.load_and_split(text_splitter)
    _ = store_db.add_documents(pages_new)

'''
# 파일에 추가 : 코드를 중복으로 실행하는 경우, 동일한 내용 중복 입력 주의 (250602)

'\nfor i in range(len(file_names)-1):\n    loader = PyMuPDFLoader(file_names[i+1])\n    pages_new = loader.load_and_split(text_splitter)\n    _ = store_db.add_documents(pages_new)\n\n'

In [5]:
# 파일로 저장된 DB 불러오기
vectorstore_path = 'store_db'

store_db_load = Chroma(
    persist_directory=vectorstore_path,
   # embedding_function=HuggingFaceEmbeddings(),
    embedding_function=embeddings_model,           # 임베딩 모델을 다르게 설정하여(모델별 벡터 차원이 다름) 발생한 문제
    collection_name="db_URN_CIS_250905",
)

In [ ]:
''' (250905 해결)
지금 코드에서는 Chroma.from_documents(..., embedding=embeddings_model)로 384차원을 사용했습니다.

그런데 similarity_search를 실행할 때는 768차원 임베딩 모델이 들어간 Chroma 인스턴스를 사용했기 때문에 dimension mismatch 발생.

해결 방법
1. 같은 임베딩 모델로 로드하기

DB를 불러올 때도 **반드시 같은 모델(384차원)**을 지정하세요.

# persist_directory 지정 후 저장
db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings_model,
    persist_directory="./chroma_db",
    collection_name="db_URN_CIS"
)
db.persist()

# 불러올 때도 같은 embedding_function 사용
store_db_load = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings_model,
    collection_name="db_URN_CIS"
)

2. 매번 새로 만들 경우 (persist 안 쓸 때)

db 객체 자체를 바로 사용하면 dimension mismatch가 일어나지 않습니다.

searchDocs = db.similarity_search("CIS(캐비테이션 초생 선속)와 URN(수중 방사 소음)", k=10)
print(searchDocs[0].page_content)


요약

지금 에러는 DB를 만들 때 사용한 임베딩 모델(384차원)과 검색할 때 쓰는 임베딩 모델(768차원)이 달라서 발생

해결책:

DB 저장(persist_directory) 후, 같은 임베딩 모델로 다시 로드해서 사용

혹은 persist 없이 db 객체를 바로 사용
'''

In [6]:
# 내용 확인

store_db_load.get()

{'ids': ['ddaf75f4-0c28-4837-9ace-f6b98704a344',
  '6570b0d7-0046-41bf-af07-a21fe54fb7c8',
  '696562dd-8d41-4dfd-aae1-1a96c2cb67e8',
  'a9c33bf1-7fa7-41b6-a423-2f3da44b9dc3',
  '43a6ed9e-f12e-4843-8f82-933dd3031748',
  'ea30d7f8-67c6-4172-969f-e99bc1a57e25',
  '53ea9d77-0324-4d23-ba33-bcc4f511b296',
  'bd8ac3f9-15b4-4140-87a3-dd63dc7a248f',
  '662c6c63-feb8-4a21-ae28-713e72c1429f',
  '29dc6ec4-dae0-4194-8828-b1ce047c4973',
  '633606bc-9f5c-4f00-8179-0087d26fc41a',
  '454fae71-bb13-4409-9bb1-fc293f3c6edd',
  '4ce15a9e-3a6b-4eb7-a8fe-d09ef228c1ea',
  'e85c0c4b-ded3-4466-b4b1-47aa051bc4e4',
  'fc834014-ff42-4985-9eff-6e1d4681c4ca',
  '9acb5feb-5bd7-4837-9f2c-fde7d48fa67a',
  '690c933c-0f74-4d3d-92ca-54ee48e01794',
  'ed16be46-69fb-40b8-9941-23cddbf595b7',
  'bf95a3e3-9479-4829-ab21-72624cc2e974',
  '2d129c6b-f243-414d-ba34-0d42d3cfe6a2',
  '2208bd57-adb2-4569-a05b-ed5a27829580',
  '711e87d0-860c-4038-a13a-b309e587e1d3',
  'd1070016-4ffa-4a48-b8cc-5b04595bc933',
  'cbdfde39-7c4a-40eb-b4bc-

In [7]:
# 유사성 측정 : 에러 메세지 이유? (250605)

question = "CIS(캐비테이션 초생 선속)와 URN(수중 방사 소음)에 대한 이론 요약"
searchDocs = store_db_load.similarity_search(question, k=10)

print(searchDocs[0].page_content)

24 
1.3.2. DEMON을 이용한 ACIS 판정 
 
 
DEMON 분석은 표적의 수중방사소음(Underwater Radiated Noise; URN)
이 증폭되는 현상을 감지하여 추진기의 정보(날개 수, 회전 수)를 추정하
는 기법이다. DEMON 분석은 Detection of Envelope Modulation On Noise의 
약자로 포락선(Envelope)를 이용한 소음신호 분석방법이다.  
수중의 임의의 위치에서 계측한 음향 신호 𝑝(𝑡)는 주파수 및 시간에 
대한 함수로 Equation 1-1과 같이 표현할 수 있다.  
 
 
𝑝(𝑡) = ∑𝐴𝑖(𝑓𝑖, 𝑡)𝑒𝑗2𝜋𝑓𝑖𝑡
∞
𝑖=−∞
 
(1-1) 
 
Equation 1-1과 같이 수중 음향 신호 𝑝(𝑡)는 무한한 주파수 신호들의 합의 
형태로 표현된다. 여기서 𝐴𝑖(𝑓𝑖, 𝑡)는 각각의 주파수 신호들의 진폭을 의
미한다. 또한, 𝑝(𝑡)는 아래의 Equation 1-2와 같이 표현할 수 있다.


In [ ]:
'''
InvalidArgumentError: Collection expecting embedding with dimension of 384, got 768
즉, ChromaDB에 저장된 벡터 차원(dimension)이 384인데, 현재 쿼리할 때 생성된 임베딩 벡터 크기가 768이라서 맞지 않아서 생기는 문제입니다.

왜 이런 문제가 생길까?
store_db_load라는 벡터스토어를 만들 때 특정 임베딩 모델을 사용했어요. (예: sentence-transformers/all-MiniLM-L6-v2 → 384차원)

그런데 지금 similarity_search 실행 시점에는 다른 임베딩 모델이 로드되어서 768차원 벡터를 만들고 있어요. (예: text-embedding-ada-002 → 1536차원, bge-base → 768차원 등)

즉, DB에 저장할 때 사용한 임베딩 모델과 검색할 때 사용한 임베딩 모델이 달라서 생긴 불일치 에러입니다.

'''

In [8]:
# retriever = db.as_retriever(search_kwargs={"k": 20})
retriever = store_db_load.as_retriever(search_kwargs={"k": 50})

#### llama 3 모델 이용

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama

# Prompt 템플릿 생성
template = '''Answer the question based only on the following context:
{context} Please answer all the answers in Korean.:

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

model = ChatOllama(model = "llama3")  

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query = "CIS(캐비테이션 초생 선속)와 URN(수중 방사 소음)에 대한 이론 요약"
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

C:\Users\wonta\AppData\Local\Temp\ipykernel_23536\2560697985.py:17: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model = "llama3")


Query: CIS(캐비테이션 초생 선속)와 URN(수중 방사 소음)에 대한 이론 요약
Answer: 캐비테이션 초생 선속(CIS, Cavitation Inception Speed)과 수중 방사 소음(URN, Underwater Radiated Noise)는 다음과 같은 이론적 요약을 갖는다.

캐비테이션 초생 선속(CIS)은 캐비테이션 초기 발생 상황부터 소음이 동반되며, 선속이 증가하면서 함께 증가한다. CIS는 추진기에서 매우 중요한 성능에 해당하는 성질로, 기포의 내파(Implosion) 특성시간은 몇 마이크로 초(Micro-second)에 이른다.

수중 방사 소음(URN)은 캐비테이션 초기 발생 상황부터 동반되는 소음으로, 추진기에서 캐비테이션이 발생하면 적함의 수동 음파탐지기(Sound Navigation and Ranging; SONAR)에 의해 위치가 노출될 수 있다. URN은 고주파수를 포함한 광대역(Broadband)의 특성을 갖는다.

따라서, CIS와 URN는 캐비테이션 초생 선속 해석과 관련된 이론적 요약을 갖는 것으로 볼 수 있다.


In [ ]:
# 상용 LLM 서비스와 같이 pdf 파일을 요약하고 발표 자료(개조식) 형태롤 출력하는 것이 가능한가? (250905)
# 위와 같은 질문에 대하여 결과를 도출하는데 15분 소요

#### LM Studio : gemma-3-4b 사용 (250909)

In [23]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# pip install langchain-openai


llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key="lm-studio",
    # model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",    # Llama 모델을 사용하는 경우 Google_API에서 반복 계속 (250821)
    # model = "lmstudio-community/gemma-2-2b-it-GGUF",
    model = "lmstudio-community/gemma-3-4b-it-GGUFF",
    # model = "mradermacher/kanana-nano-2.1b-base-GGUF",           # Llama 모델을 사용하는 경우와 동일, Google_API에서 반복 계속 (250821)
    temperature=0.2,
    streaming=False,
    callbacks=[StreamingStdOutCallbackHandler()], # 스트림 출력 콜백
)


In [24]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt 템플릿 생성
template = """
You are a summarization and presentation assistant.
Use ONLY the information in the context below: {context}
Output requirements:
Provide ONE short summary of the paper in Korean (no more than 5 sentences). 
Provide ONE presentation outline in Korean as a numbered list of slides. 
Do not repeat the summary or any slide.
Do not label slides as "Slide 1, Slide 2"; instead, just use numbers (1., 2., 3.).
End the answer with the token: <<END>>
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
# query = "CIS(캐비테이션 초생 선속)와 URN(수중 방사 소음)에 대한 발표 자료를 만들어주세요."
query = "Please summarize the paper and create presentation materials."

answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

Query: Please summarize the paper and create presentation materials.
Answer: Okay, here’s a summary of the paper in Korean, followed by a presentation outline as requested.

**Summary (Korean):**

본 논문은 선박 추진기의 캐비테이션 초생 속도(CIS)를 예측하기 위한 전산유체역학(CFD) 해석 시스템 개발에 대해 다룬다.  날개 끝 보텍스 캐비테이션을 중심으로, 기포동역학적 모델링과 레이leigh-Plesset 방정식을 활용하여 CIS를 정확하게 예측하는 방법을 제시한다.  개발된 시스템은 다양한 수중 구조물에 대한 해석 결과를 통해 유효성을 검증했으며, 잠수함 추진기의 캐비테이션 초생 선속을 평가하는데 기여할 것으로 기대된다. 본 연구는 전산유체역학(CFD) 해석의 활용 가능성을 보여주는 중요한 사례이다.

---

**Presentation Outline (Korean)**

1.  **서론:** 본 연구의 배경 및 목적 소개 - 캐비테이션 초생 선속의 중요성, 해군 함정 성능 개선을 위한 필요성
2.  **이론적 배경:** 캐비테이션 현상에 대한 이해 - 캐비테이션 정의, 발생 메커니즘 (Rayleigh-Plesset 방정식, 기포동역학)
3.  **전산유체역학 해석 시스템 구축:** 개발된 시스템의 구성 요소 및 특징 설명 - CFD 해석 방법론, 날개 끝 보텍스 캐비테이션 모델링
4.  **해석 결과 분석 및 검증:** 다양한 수중 구조물에 대한 해석 결과 제시 및 실험 데이터와의 비교 – DTMB 함정 적용 사례, 소음 예측 정확도 향상 효과
5.  **향후 연구 방향 제안:** 잠수함 추진기 캐비테이션 초생 선속 해석 시스템 고도화 및 해군 함정 성능 개선 방안 모색

---

**Presentation Outline (Korean)**

1.  **서론 (Introduction):** 선박 추

In [ ]:
# LM Studio > Load > Context Length : 10200으로 설정 -> 에러 메세지 확인 후, 좀 더 큰 값으로 입력할 것(250910)
# Step 1에서 무한 반복 문제점 발생 (250910)

In [27]:
# 템플릿 옵션 변경 #3

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt 템플릿 생성
template = """
You must answer using ONLY the information in the context below. 
Do NOT use any outside knowledge.

Context:
{context}

Task:
1) Create a presentation outline in Korean for PPT slides. 
   - Each slide must include: 
     • Title 
     • 3–4 bullet points 
     • 1–2 sentence speaker note
2) Produce each part exactly ONCE and do not repeat.
3) End your answer with the token: <<END>>

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
# query = "CIS(캐비테이션 초생 선속)와 URN(수중 방사 소음)에 대한 발표 자료를 만들어주세요."
query = "Please create presentation materials."

answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

Query: Please create presentation materials.
Answer: Okay, here’s a detailed presentation outline in Korean for PPT slides focusing on the provided context (cavitation inception analysis), following your instructions.

**Presentation Outline – 캐비테이션 초생 선속 분석 (Cavitation Inception Speed Analysis)**

**(Slide 1: Title Slide)**

*   Title: 캐비테이션 초생 선속 분석 및 시스템 개발 (Analysis and System Development of Cavitation Inception Speed)
*   Bullet Points:
    • 연구 목표 제시 (Presentation of Research Objectives)
    • 핵심 기술 소개 (Introduction of Core Technologies)
    • 결과 요약 (Summary of Results)
*   Speaker Note: “본 발표에서는 선박 추진기의 캐비테이션 초생 선속을 예측하기 위한 새로운 분석 시스템 개발에 대해 설명합니다.  이 시스템은 기포동역학적 모델링과 전산 유체 역학(CFD) 기술을 통합하여 활용됩니다.”

**(Slide 2: Cavitation Overview – 캐비테이션 개요)**

*   Title: 캐비테이션 현상 이해 (Understanding the Cavitation Phenomenon)
*   Bullet Points:
    • 캐비테이션 정의 (Definition of Cavitation)
    • 캐비테이션 유형 분류 (Classification of Cavitation Types - 이동, 고정, 진동)
    • 캐비테이션 발생 요인 (Factors Contributing to 

In [ ]:
# LM Studio > Load > Context Length : 10300으로 설정 -> 에러 메세지 확인 후, 좀 더 큰 값으로 입력할 것(250910)
# Step 1에서 무한 반복을 해결하기 위해 템플릿 수정 (250910)

# 실제 논문의 제목과 LLM 이 도출한 발표자료의 제목이 다른 점을 주목하자 (250910)